In [ ]:
import sys,os
sys.path.append("..")
import django
django.setup()
from crawlers.models import *
import pandas as pd
import datetime
from components.price_feature import GetAdj
import numpy as np
from components.backend_api import DataFilter
from components.backtest import Backtest
import matplotlib.pyplot as plt


# Strategy

In [ ]:
rev_data=DataFilter(model=MonthlyRevenueTW,fields=['stock_id','date','this_month_rev','cp_last_year_rev','cm_this_month_rev'],start_date='2012-01-10').get_pivot()
cp_last_year_rev=rev_data['cp_last_year_rev']
cp_last_year_rev.index = pd.to_datetime(cp_last_year_rev.index).shift(5, "d")
rev=rev_data['this_month_rev']
rev.index = pd.to_datetime(rev.index).shift(5, "d")
rev3=rev.rolling(3, min_periods=3).mean()
rev12=rev.rolling(12, min_periods=12).mean()
revg3=round((rev3-rev3.shift(12))/(rev3.shift(12))*100,2)#近3月平均營收年增率
revg12=round((rev12-rev12.shift(12))/(rev12.shift(12))*100,2)#近12月平均營收年增率
revg12


In [ ]:
cond1=rev3/rev12>1.05
cond1

In [ ]:
cond2=(cp_last_year_rev>30)&(cp_last_year_rev<100)
cond2

In [ ]:
# data=[]
# freq=3
# date_set=cp_last_year_rev.index[freq:]
# for i in range(freq,len(cp_last_year_rev)):
#     dgg=(cp_last_year_rev.iloc[i-freq:i]>20).sum()==freq
#     data.append(dgg)
# cond3=pd.DataFrame(data,date_set)
# cond3

# 價格濾網

In [ ]:
price_data=DataFilter(model=StockPriceTW,fields=['stock_id','date','close_price','turnover_vol'],start_date='2012-01-10',market=['sii', 'otc']).get_pivot()
df=price_data['close_price']
start_date=df.index[0]
last_date = df.index[-1]
date_range = pd.date_range(start=start_date, end=last_date)
df = df.reindex(date_range).fillna(method='ffill').dropna(how='all')
df2=df.reindex(cond1.index)
cond4=(df2>10)&(df2<=150)
cond4

# 交易量濾網

In [ ]:
turnover_vol=price_data['turnover_vol']
turnover_vol=turnover_vol.rolling(5, min_periods=5).mean()
turnover_vol = turnover_vol.reindex(date_range).fillna(method='ffill').dropna(how='all')
turnover_vol=turnover_vol.reindex(cond1.index)
cond5=turnover_vol>500000
cond5


In [ ]:

# cond5 = rev.iloc[-1] == rev.iloc[-12:].max()
# cond5=cond5[cond5 == True]
# len(cond5)

# data=[]
# date_set=rev.index[12:]
# for i in range(12,len(rev)):
#     dgg=rev.iloc[i-12:i]
#     max_cond = dgg.iloc[-1] == dgg.max()
#     data.append(max_cond)
# cond5=pd.DataFrame(data,date_set)
# cond5

# freq=3
# data=[]
# date_set=cp_last_year_rev.index[freq:]
# for i in range(freq,len(cp_last_year_rev)):
#     _revg3=revg3.iloc[i-freq:i]
#     _revg12=revg12.iloc[i-freq:i]
#     _cond1=_revg3.iloc[-1]>_revg12.iloc[-1]
#     _cond2=_revg3.iloc[-freq]<_revg12.iloc[-freq]
#     data.append(_cond1&_cond2)
# cond5=pd.DataFrame(data,date_set)
# cond5



In [ ]:
strategy=(cond1&cond2&cond4&cond5)
strategy

In [ ]:
df=Backtest(start_date='2012-01-10',strategy=strategy)
df=df.report()
df

In [ ]:
pd.Series(df['benchmark'].cumprod(),df['date']).plot(label='benchmark')
pd.Series(df['cumprod_result'],df['date']).plot(label='return')
plt.legend()

In [ ]:
pd.Series(df['result']-df['result'].mean(),df['date']).plot(kind='hist')

In [ ]:
ss=strategy.iloc[-2].dropna()
len(ss[ss])